Author: Rui ZHANG 

Department: EIT

E-mail: cheryl_zhangrui@163.com / rui.zhang1@etu.unice.fr


# **0 Readme**



This notebook gives the example of using the laptop camera and opencv-python Haar Feature-based cascade face detector of Adaboost classifier to detect human face. The notebook is structured as follows:

1. Import Libraries
2. Helper Functions
3. Face Detection (Simple)
4. Face Detection (Accelerated)

The explanation of the method, steps and result analysis will follow the title of that part.

To run the code, please run the cells in sequence.

# **1 Import Libraries**

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np
from PIL import Image
import io
import cv2

# **2 Helper Functions**

### **2.1 Camera calling and display**

Javascript code to call the camera and display the video and image in Google Coloab (from teacher)

In [ ]:
def VideoCapture():
  js = Javascript('''
    async function create(){
      div = document.createElement('div');
      document.body.appendChild(div);

      video = document.createElement('video');
      video.setAttribute('playsinline', '');

      div.appendChild(video);

      stream = await navigator.mediaDevices.getUserMedia({video: {facingMode: "environment"}});
      video.srcObject = stream;

      await video.play();

      canvas =  document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);

      div_out = document.createElement('div');
      document.body.appendChild(div_out);
      img = document.createElement('img');
      div_out.appendChild(img);
    }

    async function capture(){
        return await new Promise(function(resolve, reject){
            pendingResolve = resolve;
            canvas.getContext('2d').drawImage(video, 0, 0);
            result = canvas.toDataURL('image/jpeg', 0.8);
            pendingResolve(result);
        })
    }

    function showimg(imgb64){
        img.src = "data:image/jpg;base64," + imgb64;
    }

  ''')
  display(js)

def byte2image(byte):
  jpeg = b64decode(byte.split(',')[1])
  im = Image.open(io.BytesIO(jpeg))
  return np.array(im)

def image2byte(image):
  image = Image.fromarray(image)
  buffer = io.BytesIO()
  image.save(buffer, 'jpeg')
  buffer.seek(0)
  x = b64encode(buffer.read()).decode('utf-8')
  return x



### **2.2 Other helper functions**

Includes the following functions:

**clock()**: timing function

**draw_rects(img, rect, color)**: function to draw rectangles in the image, to show the detection result

**draw_string(dst, target, s)**: function to put a string in the image, to show the detection time

**detect(img, cascade)**: use the classifier to detect face and return the result

**expand_rect(rect, pix)**: expand a given rect with certain pixels to be the detection ROI (region of interest)

In [ ]:
def clock():
  """
  Get the time (calculated by clock cycles / clock frequency).
  From opencv examples common.py.
  :return: Present timestamp (in seconds).
  """
  return cv2.getTickCount() / cv2.getTickFrequency()


def draw_rects(img, rects, color):
  """
  Draw rectangles in a given image using the given rectangle list.
  From opencv examples facedetect.py.
  :param img: the image on which to be drawn the rectangles
  :param rect: a list of rectangles' coordinates, in the order [x1, y1, x2, y2]
  :param color: draw color
  :return: None
  """
  for rect in rects:
    [x1, y1, x2, y2] = rect
    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)


def draw_str(dst, target, s):
  """
  Put a string of text on the given image.
  From opencv examples common.py.
  :param dst: string position
  :param target: the image to be put text on
  :param s: the string to be shown
  :return: None
  """
  x, y = target
  # Strings with 1 pixel translation in x and y and different colors to make the text more obvious to be seen
  cv2.putText(dst, s, (x+1, y+1), cv2.FONT_HERSHEY_PLAIN, 1.0, (0, 0, 0), thickness = 2, lineType=cv2.LINE_AA)
  cv2.putText(dst, s, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.0, (255, 255, 255), lineType=cv2.LINE_AA)


def detect(img, cascade):
  """
  Detect the face in the input image using the cascade detector.
  From opencv examples facedetect.py.
  :param img: the input image to be detected
  :param cascade: the classifier
  :return: the detected faces, in a list of rectangles
  """
  # Detect faces of different size in the input image, reduce the image size by 30% when constructing the image 
  # pyramid, set the minimum face to be detected at size 30x30
  rects = cascade.detectMultiScale(img, scaleFactor=1.3, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
  # No face detected in the input image
  if len(rects) == 0:
    return []
  # The detected result is given as [x, y, w, h], change it to [x1, y1, x2, y2]=[x, y, x+w, y+h] for convenience
  rects[:,2:] += rects[:,:2]
  return rects


def expand_rect(rect, pix):
  """
  Expand the given rectangle by expanding outwards 'pix' pixels at each side
  :param rect: the coordinates of the original rectangle [x1, y1, x2, y2]
  :param pix: the pixels to be expanded at each side
  :return: the coordinates of the expanded rectangle [x1', y1', x2', y2']
  """
  expanded = [rect[0]-pix, rect[1]-pix, rect[2]+pix, rect[3]+pix]
  return [int(item) for item in expanded]

# **3 Face Detection (Simple)**

**Steps:**

The process of the face detection in the whole image: 

1. create a real-time video stream
2. load the cascade classifier
3. capture the present frame and preprocessing (format and color transformation)
4. detect the face in the image
5. show the result and detection time, go to step 3 until stop.

Detailed explanations of each step are within the code.

**Result:**

This simple version of face detection time ranges from 50~80ms per detection depending on the background.

In [ ]:
# Create a real-time video object using the javascript code.
VideoCapture()
eval_js('create()')

# Load the trained Haar Feature-based cascade classifier from the xml file, which contains the features of the
# 38 classifiers, and their corresponding weights
cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

while True:
  # Capture the present image
  byte = eval_js('capture()')

  # Format transformation, transform the image to gray
  im = byte2image(byte)
  gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

  # Start time
  t = clock()

  # Detection
  rects = detect(gray, cascade)

  # End time
  dt = clock() - t

  # Draw the detected rectangles, put the detection time (ms), show the image
  draw_rects(im, rects, (255, 0, 0))
  draw_str(im, (20, 20), 'time: %.1f ms' % (dt * 1000))
  eval_js('showimg("{}")'.format(image2byte(im)))

MessageError: ignored

# **4 Face Detection (Accelerated)**

**Steps:**

The accelerated version of face detection:

1. create a real-time video stream
2. load the cascade classifier
3. capture an image and preprocessing (format and color transformation)
4. if RESTART==True, restart detecting a face in the whole image; if RESTART=False, detect a face in the ROI (region of interest, which is an expanded area of the former detection result)
5. detect the face
6. update the ROI and RESTART according to the detection result
7. show the result and detection time, go to step 3 until stop.

Detailed explanations of each step are within the code.

**Result:**

The accelerated version achives around half of the detecting time than before (from 60ms to 30ms on my laptop, may change according to the background, the network, the computer camera, etc.) when detecting a face in the ROI (this can be speeded up by decreasing the pixels to be expanded around the ROI, but when expand less, the possibility to detect a fast-moving face in the ROI also decrease, thus leads to more restarts), and the same detection time when restart detection using the whole image as before.

In [ ]:
# Create a real-time video object using the javascript code.
VideoCapture()
eval_js('create()')

# Reset the RESTART flag to be True, which means a face is not detected in the former frame or the start of the
# detection, we need to restart to detect the face using the whole image
RESTART = True

# Load the trained Haar Feature-based cascade classifier from the xml file, which contains the features of the
# 38 classifiers, and their corresponding weights
cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_alt.xml")

while True:
    # Capture the present frame, transform its format and to gray image
    byte = eval_js('capture()')
    im = byte2image(byte)
    gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

    # Clock start
    t = clock()

    ## If RESTART==True, detect in the whole image
    if RESTART:
        det_rect = detect(gray, cascade)
        ## If no face is detected, reset RESTART=True, continue to next round detection
        if len(det_rect) == 0:
            RESTART = True
            continue

        ## If a face is detected:
        # 1) set RESTART=False, which means next round we don't have to detect in the whole image
        # 2) select the first detected rectangle, expand it to be the roi to be detected in the next round
        # 3) draw the rectangle
        else:
            dt = clock() - t
            RESTART = False
            rect = det_rect[0]
            roi = expand_rect(rect, 50)
            draw_rects(im, [rect], (255, 0, 0))

    ## If RESTART=False, detect in the roi
    else:
        det_rect = detect(gray[roi[1]:roi[3], roi[0]:roi[2]], cascade)
        ## If no face is detected, reset RESTART, continue to next round detection
        if len(det_rect) == 0:
            RESTART = True
            continue

        ## If a face is detected:
        # 1) select the first detected ranctangle to move on
        # 2) map the coordinates in the roi to the original image by adding the top left point's coordinates of
        #   the roi to each coordinates of the detected result (as the detection result is within the roi)
        # 3) draw the mapped rectangle in original image
        # 4) expand the rectangle to be the roi in next round
        else:
            dt = clock() - t
            rect = det_rect[0]
            mapped = [rect[0]+roi[0], rect[1]+roi[1], rect[2]+roi[0], rect[3]+roi[1]]
            draw_rects(im, [mapped], (255, 0, 0))
            roi = expand_rect(mapped, 50)

    # Put the string of elapsed time (ms), show the image
    draw_str(im, (20, 20), "time: %.1f ms" % (dt * 1000))
    eval_js('showimg("{}")'.format(image2byte(im)))

MessageError: ignored